In [9]:
import sys
sys.path.append('../')  # Adjust the path depending on the actual path to folder1

import os
import torch
import pandas as pd
from torch import nn
from torch.nn import functional as F
from tqdm import tqdm
from defender.utils.load_models import load_model
from defender.utils.feature_extraction import extract_features

In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

# Load the Models
rf_model_path = "../models/rf_ember_subset_2.joblib"
malconv_path = "../models/malconv_v2.pt"
rf_model = load_model("RF", rf_model_path)
malconv = load_model("Malconv", malconv_path)

In [ ]:
# Run on testset
basePath = "../test/datasets"
all_results = []


for folder in os.listdir(basePath):
    if "mw" in folder:
        label = 1
    else:
        label = 0
    results = []
    folderPath = os.path.join(basePath, folder)
    fileCount = 0
    if os.path.isfile(folderPath):
        continue
    for file in tqdm(os.listdir(folderPath)):
        filePath = os.path.join(folderPath, file)
        # break
        with open(filePath, "rb") as f :
            data = f.read()
        try:
            features_data, header = extract_features(data)
        except:
            results.append([folder+"_"+file] + [folder] + [label] + [1.0, 0.0, 0.0])
            continue
        input = torch.tensor(header).unsqueeze(0).to(device)
        y_pred = rf_model.predict_proba(features_data)
        with torch.no_grad():
            y_pred2 = torch.sigmoid(malconv(input))
            features = torch.tensor(features_data, dtype=torch.float32)
            results.append([folder+"_"+file] + [folder] + [label] +y_pred.tolist()[0] + y_pred2.tolist() )
        # break
    all_results += results


In [10]:
df = pd.DataFrame(all_results, columns=["FileName", "Dataset", "Label", "y_pred0_0", "y_pred0_1", "y_pred1"])
df.to_csv("all_reults.csv")
df.head()

,FileName,Dataset,Label,y_pred0_0,y_pred0_1,y_pred1
0,gw2_0000,gw2,0,0.288106,0.711894,0.817315
1,gw3_0000,gw3,0,0.960453,0.039547,0.046224
2,gw4_0000,gw4,0,0.087363,0.912637,0.999226
3,gw5_0000,gw5,0,0.128406,0.871594,0.999988
4,gw6_0000,gw6,0,0.902080,0.097920,0.259076
